In [1]:
!pip install shap

     |████████████████████████████████| 276kB 4.9MB/s 
  Created wheel for shap: filename=shap-0.35.0-cp36-cp36m-linux_x86_64.whl size=394152 sha256=b46caca2eaeea2faf4078446260b26815f6e2a6fe3eb8834a806077408e2d57f
  Stored in directory: /root/.cache/pip/wheels/e7/f7/0f/b57055080cf8894906b3bd3616d2fc2bfd0b12d5161bcb24ac
Successfully built shap


In [0]:
import numpy as np                   # array, vector, matrix calculations
import pandas as pd                  # DataFrame handling
import shap                          # for consistent, signed variable importance measurements
import lightgbm as lgb                # gradient boosting machines (GBMs)

import matplotlib.pyplot as plt      # plotting
pd.options.display.max_columns = 999 # enable display of all columns in notebook

# enables display of plots in notebook
%matplotlib inline

np.random.seed(12345)                # set random seed for reproducibility

In [0]:
# import XLS file
path = 'https://github.com/firmai/random-assets-two/blob/master/xaib/default_of_credit_card_clients.xls?raw=true'
data = pd.read_excel(path,
                     skiprows=1) # skip the first row of the spreadsheet

# remove spaces from target column name 
data = data.rename(columns={'default payment next month': 'DEFAULT_NEXT_MONTH'}) 

In [4]:
# assign target and inputs for GBM
y = 'DEFAULT_NEXT_MONTH'
X = [name for name in data.columns if name not in [y, 'ID']]
print('y =', y)
print('X =', X)

y = DEFAULT_NEXT_MONTH
X = ['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6']


In [0]:
best_params = {'bagging_fraction': 0.2,
 'bagging_seed': 1,
 'objective': 'binary',

 'verbose': -1} 

In [0]:
# execute split, set random state for reproducibility
train = data.sample(frac=0.7,random_state=1) # 70%/30% train/test split
test = data.drop(train.index).set_index("ID",drop=True)
train = train.set_index("ID",drop=True)

In [0]:
train_first = train.sample(frac=0.5,random_state=1)
train_second = train.drop(train_first.index)

In [9]:

from sklearn.metrics import roc_auc_score

# execute split, set random state for reproducibility

d_train_first = lgb.Dataset(train_first.drop([y],axis=1), label=train_first[y])
#d_train_second = lgb.Dataset(train_second.drop([y],axis=1), label=train_second[y])
model = lgb.train(best_params, d_train_first, verbose_eval=1000)
y_pred_second = model.predict(train_second.drop([y],axis=1))
print('ROC AUC {}'.format(roc_auc_score(train_second[y], y_pred_second)))

y_pred_test = model.predict(test.drop([y],axis=1))
print('ROC AUC {}'.format(roc_auc_score(test[y], y_pred_test)))


ROC AUC 0.7713549212252115
ROC AUC 0.7810723558640761


In [0]:
# Crete new dataframe to experience all tranformations
data_n = data.set_index("ID",drop=True).copy()

In [11]:
# Identify interactions
import math

input_row = len(train_first) 
sampled = min(1, (1/math.log(input_row, 3))*np.exp((2/math.log(input_row, 10000000)))/20)

# sampled = sampled *2

explainer = shap.TreeExplainer(model)

single = explainer.shap_values(train_first.drop([y],axis=1)[:int(input_row*sampled)].values)

inter = shap.TreeExplainer(model).shap_interaction_values(train_first.drop([y],axis=1)[:int(input_row*sampled*sampled)].values)
# Same as model

Setting feature_perturbation = "tree_path_dependent" because no background data was given.
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


In [12]:
is_true = False
inter_portion = 0.5

shap_fram = pd.DataFrame(single[0][:,:], columns=X)
shap_fram = shap_fram.mean().sort_values().to_frame()
shap_new = shap_fram[0].abs().sort_values(ascending=is_true)
main_ft = shap_new[shap_new.cumsum().sub((shap_new.sum()*inter_portion)).le(0)]


# for is_true in [False, True]:
#   for inter_portion in list(np.linspace(0.1,1,10,endpoint=False)):

print(str(is_true) + " " + str(inter_portion))
# create new contributive features
def main_calc(new_df,main_ft):
  df_square = new_df[list(main_ft.index)]
  df_square = df_square - df_square.min()
  sqr_name = [str(fa)+"_POWER_2" for fa in df_square.columns]
  log_p_name = [str(fa)+"_LOG_p_one_abs" for fa in df_square.columns]
  rec_p_name = [str(fa)+"_RECIP_p_one" for fa in df_square.columns]
  sqrt_name = [str(fa)+"_SQRT_p_one" for fa in df_square.columns]

  df_sqr = pd.DataFrame(np.power(df_square.values, 2),columns=sqr_name, index=new_df.index)
  df_log = pd.DataFrame(np.log(df_square.add(1).abs().values),columns=log_p_name, index=new_df.index)
  df_rec = pd.DataFrame(np.reciprocal(df_square.add(1).values),columns=rec_p_name, index=new_df.index)
  df_sqrt = pd.DataFrame(np.sqrt(df_square.abs().add(1).values),columns=sqrt_name, index=new_df.index)

  dfs = [df_sqr, df_log, df_rec, df_sqrt]

  df_connect=  pd.concat(dfs, axis=1)

  return df_connect

main_c = main_calc(data_n,main_ft)

## Finding the top interactions

df_start = pd.DataFrame(np.abs(np.mean(inter ,axis=0)),columns=X, index=X)

#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)
sol = (df_start.where(np.triu(np.ones(df_start.shape), k=1).astype(np.bool))
                .stack().sort_values(ascending=is_true))
                
dab = sol[sol.cumsum().sub((sol.sum()*inter_portion)).le(0)]

list_one = [da[0] for da in dab.index]
list_two = [da[1] for da in dab.index]


## Creating new formulaic interactions

def inter_cal(list_one, list_two,new_df):

  new_df = new_df - new_df.min() ## Don't want dividion zero
  mult = [str(ra)+"_X_"+str(ba) for ra, ba in zip(list_one, list_two)]
  div = [str(ra)+"_DIV_"+str(ba) for ra, ba in zip(list_one, list_two)]
  print("len one " + str(len(list_one)) )
  print("len two " + str(len(list_two)) )
  inter_mult = pd.DataFrame(new_df[list_one].values*new_df[list_two].values, columns=mult, index=new_df.index)
  div_p_one = pd.DataFrame(new_df[list_one].values/new_df[list_two].add(1).values, columns=div, index=new_df.index)

  df_one = pd.concat((inter_mult,div_p_one), axis=1)

  return df_one


inter_md = inter_cal(list_one, list_two,data_n)

## Creating sum interactions

def inter_sum(df, sol):
  ungroup = sol.to_frame().reset_index()
  group = ungroup.groupby("level_0").mean().sort_values(by=0,ascending=is_true)
  gr = group[0]
  gr = gr[gr.cumsum().sub((gr.sum()*inter_portion)).le(0)]
  list_cols = []
  for gra in gr.index:
    summer = ungroup[(ungroup["level_0"]==gra)|(ungroup["level_1"]==gra)].sort_values(0,ascending=is_true).set_index(['level_0', 'level_1'])
    suum = summer[0]
    suum = suum[suum.cumsum().sub((suum.sum()*inter_portion)).le(0)]
    list_one = [da[0] for da in suum.index]
    list_two = [da[1] for da in suum.index]
    list_one.extend(list_two)
    lit = list(set(list_one))
    name = "_ADD_".join(lit)
    df[name] = df[lit].sum(axis=1)
    list_cols.append(name)
  return df[list_cols]

inter_s = inter_sum(data_n, sol)


pdList = [data_n, main_c, inter_md, inter_s ] #inter_md, main_c, inter_s
new_df = pd.concat(pdList, axis=1)

## First One
# execute split, set random state for reproducibility
train = new_df.sample(frac=0.7,random_state=1) # 70%/30% train/test split
test = new_df.drop(train.index)

train_first = train.sample(frac=0.5,random_state=1)
train_second = train.drop(train_first.index)

d_train_first = lgb.Dataset(train_first.drop([y],axis=1), label=train_first[y])
#d_train_second = lgb.Dataset(train_second.drop([y],axis=1), label=train_second[y])

model = lgb.train(best_params, d_train_first, verbose_eval=1000)
y_pred_second = model.predict(train_second.drop([y],axis=1))
print('ROC AUC {}'.format(roc_auc_score(train_second[y], y_pred_second)))

y_pred_test = model.predict(test.drop([y],axis=1))
print('ROC AUC {}'.format(roc_auc_score(test[y], y_pred_test)))



False 0.5
len one 29
len two 29
ROC AUC 0.7726681725199329
ROC AUC 0.7778544047490384


In [14]:
explainer = shap.TreeExplainer(model)
single = explainer.shap_values(train_first.drop([y],axis=1)[:int(input_row*sampled)].values)
shap_fram = pd.DataFrame(single[0][:,:], columns=train_first.drop([y],axis=1).columns)
shap_fram = shap_fram.mean().sort_values().to_frame()
shap_new = shap_fram[0].abs().sort_values(ascending=is_true)


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
LightGBM binary classifier with TreeExplainer shap values output has changed to a list of ndarray


,PAY_0,LIMIT_BAL_X_PAY_AMT1,PAY_AMT3_X_PAY_AMT4,BILL_AMT2,PAY_3_X_BILL_AMT1,BILL_AMT1_ADD_BILL_AMT4_ADD_BILL_AMT3,BILL_AMT1_ADD_BILL_AMT4_ADD_BILL_AMT3,BILL_AMT1_DIV_PAY_AMT1,BILL_AMT4_DIV_PAY_AMT5,BILL_AMT1_DIV_PAY_AMT2,BILL_AMT1_X_BILL_AMT4,BILL_AMT3_DIV_BILL_AMT4,BILL_AMT1,PAY_6,LIMIT_BAL_DIV_BILL_AMT5,PAY_5
ID,,,,,,,,,,,,,,,,
1,2,0,0,3102,169493,4602,4602,169493.000000,170000.000000,245.642029,28813810000,0.929130,3913,-2,0.122948,-2
2,-1,0,1000000,1725,336524,8636,8636,168262.000000,173272.000000,168.093906,29155093264,0.923087,2682,2,1.297323,0
3,0,121440000,1000000,14027,389638,57129,57129,128.254773,184.146853,129.792805,35911181089,0.926714,29239,0,0.830884,0
4,0,80000000,1320000,48233,425140,124595,124595,106.231884,185.340187,105.232673,42155606980,1.041550,46990,0,0.362667,0
5,-1,80000000,90000000,5670,174197,65392,65392,87.054973,276.724638,4.748841,33261175180,1.011302,8617,0,0.398085,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29996,0,1785000000,15244141,192815,709056,485317,485317,41.704270,51.590482,17.725514,91469642112,1.417139,188948,0,1.865473,0
29997,-1,257180000,1160742,1828,167263,14164,14164,91.002720,178979.000000,47.423589,29936564477,0.898234,1683,0,1.618029,0
29998,4,0,92400000,3356,676580,27201,27201,169145.000000,95.391304,169145.000000,32286059310,0.838343,3565,0,0.196238,0


In [22]:
main_ft.head(20)

PAY_0                                                 0.031307
LIMIT_BAL_X_PAY_AMT1                                  0.018804
PAY_AMT3_X_PAY_AMT4                                   0.017929
BILL_AMT2                                             0.017352
PAY_3_X_BILL_AMT1                                     0.015120
BILL_AMT1_ADD_BILL_AMT4_ADD_BILL_AMT3                 0.014086
BILL_AMT1_DIV_PAY_AMT1                                0.013410
BILL_AMT4_DIV_PAY_AMT5                                0.012612
BILL_AMT1_DIV_PAY_AMT2                                0.012019
BILL_AMT1_X_BILL_AMT4                                 0.011474
BILL_AMT3_DIV_BILL_AMT4                               0.010926
BILL_AMT1                                             0.010903
PAY_6                                                 0.010115
LIMIT_BAL_DIV_BILL_AMT5                               0.009578
PAY_5                                                 0.009193
PAY_0_DIV_PAY_2                                       0

In [26]:
shap_new[shap_new.cumsum().sub((shap_new.sum()*0.8)).le(0)]

PAY_0                                                 0.031307
LIMIT_BAL_X_PAY_AMT1                                  0.018804
PAY_AMT3_X_PAY_AMT4                                   0.017929
BILL_AMT2                                             0.017352
PAY_3_X_BILL_AMT1                                     0.015120
BILL_AMT1_ADD_BILL_AMT4_ADD_BILL_AMT3                 0.014086
BILL_AMT1_DIV_PAY_AMT1                                0.013410
BILL_AMT4_DIV_PAY_AMT5                                0.012612
BILL_AMT1_DIV_PAY_AMT2                                0.012019
BILL_AMT1_X_BILL_AMT4                                 0.011474
BILL_AMT3_DIV_BILL_AMT4                               0.010926
BILL_AMT1                                             0.010903
PAY_6                                                 0.010115
LIMIT_BAL_DIV_BILL_AMT5                               0.009578
PAY_5                                                 0.009193
PAY_0_DIV_PAY_2                                       0

In [67]:
# input_row = len(train_second) 
# sampled = min(1, (1/math.log(input_row, 3))*np.exp((2/math.log(input_row, 10000000)))/20)


for bar in [.99, .95, .90, .80,.7,.6]:

  main_ft = shap_new[shap_new.cumsum().sub((shap_new.sum()*bar)).le(0)]
  print(len(list(main_ft.index)))
  new_df_2 = new_df[list(main_ft.index)].copy()
  new_df_2[y] = new_df[y]

  train = new_df_2.sample(frac=0.7,random_state=1) # 70%/30% train/test split
  test = new_df_2.drop(train.index)

  train_first = train.sample(frac=0.5,random_state=1)
  train_second = train.drop(train_first.index)

  #d_train_first = lgb.Dataset(train_first.drop([y],axis=1), label=train_first[y])
  #model = lgb.train(best_params, d_train_first, verbose_eval=1000)

  d_train_first = lgb.Dataset(train_first.drop([y],axis=1), label=train_first[y])
  model = lgb.train(best_params, d_train_first, verbose_eval=1000)

  y_pred_second = model.predict(train_second.drop([y],axis=1))
  print('ROC AUC {}'.format(roc_auc_score(train_second[y], y_pred_second)))

  y_pred_test = model.predict(test.drop([y],axis=1))
  print('ROC AUC {}'.format(roc_auc_score(test[y], y_pred_test)))




77


KeyError: ignored

In [37]:
neigh.predict_proba(test[X])[:, 0]

array([0.33333333, 0.33333333, 1.        , ..., 1.        , 0.66666667,
       0.66666667])

In [34]:
neigh.predict_proba(test[X])

array([[0.33333333, 0.66666667],
       [0.33333333, 0.66666667],
       [1.        , 0.        ],
       ...,
       [1.        , 0.        ],
       [0.66666667, 0.33333333],
       [0.66666667, 0.33333333]])

In [96]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

train = data.sample(frac=0.7,random_state=1) # 70%/30% train/test split
test = data.drop(train.index).set_index("ID",drop=True)
train = train.set_index("ID",drop=True)

neigh = RandomForestClassifier()
neigh.fit(train.drop([y],axis=1), train[y])

print('ROC AUC {}'.format(roc_auc_score(test[y], neigh.predict_proba(test.drop([y],axis=1))[:, 1])))


ROC AUC 0.7686945780583906


In [0]:
data_n = data_n.T.drop_duplicates().T

In [0]:
pdList = [data_n, main_c, inter_md, inter_s ] #inter_md, main_c, inter_s
new_df = pd.concat(pdList, axis=1)


In [0]:
new_df = new_df.T.drop_duplicates().T

In [95]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

train = new_df.sample(frac=0.7,random_state=1) # 70%/30% train/test split
test = new_df.drop(train.index)

neigh = RandomForestClassifier()
neigh.fit(train.drop([y],axis=1), train[y])

print('ROC AUC {}'.format(roc_auc_score(test[y], neigh.predict_proba(test.drop([y],axis=1))[:, 1])))



ROC AUC 0.7630901550072805


In [75]:
train[y]

,DEFAULT_NEXT_MONTH,DEFAULT_NEXT_MONTH
ID,,
10748,0,0
12574,1,1
29677,0,0
8857,1,1
21099,0,0
...,...,...
27956,0,0
27108,0,0
26,0,0


In [74]:
from sklearn.neighbors import KNeighborsClassifier

train = new_df.sample(frac=0.7,random_state=1) # 70%/30% train/test split
test = new_df.drop(train.index)

neigh = LogisticRegression()
neigh.fit(train.drop([y],axis=1), train[y])

print('ROC AUC {}'.format(roc_auc_score(test[y], neigh.predict_proba(test.drop([y],axis=1))[:, 1])))



ValueError: ignored

['BILL_AMT6', 'BILL_AMT4', 'BILL_AMT5', 'PAY_AMT6']

In [0]:
gr.index

Index(['BILL_AMT5', 'BILL_AMT4', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT1'], dtype='object', name='level_0')

In [0]:
target = y

In [0]:
inter_mult.head()

,BILL_AMT5_X_BILL_AMT6,BILL_AMT4_X_BILL_AMT5,BILL_AMT3_X_BILL_AMT4,BILL_AMT4_X_PAY_AMT5,BILL_AMT2_X_BILL_AMT3,BILL_AMT1_X_BILL_AMT6,BILL_AMT5_X_PAY_AMT6,BILL_AMT1_X_BILL_AMT2,BILL_AMT5_X_PAY_AMT1,BILL_AMT2_X_PAY_AMT1,BILL_AMT3_X_BILL_AMT5,BILL_AMT2_X_PAY_AMT3,PAY_AMT1_X_PAY_AMT3,PAY_4_X_BILL_AMT1,BILL_AMT5_X_PAY_AMT4,BILL_AMT3_X_BILL_AMT6,BILL_AMT3_X_PAY_AMT3,BILL_AMT2_X_BILL_AMT5,BILL_AMT4_X_PAY_AMT3,PAY_AMT2_X_PAY_AMT5,PAY_AMT3_X_PAY_AMT4,PAY_AMT3_X_PAY_AMT6,LIMIT_BAL_X_BILL_AMT1,PAY_AMT2_X_PAY_AMT3,PAY_0_X_PAY_3,BILL_AMT2_X_BILL_AMT4,BILL_AMT5_DIV_BILL_AMT6,BILL_AMT4_DIV_BILL_AMT5,BILL_AMT3_DIV_BILL_AMT4,BILL_AMT4_DIV_PAY_AMT5,BILL_AMT2_DIV_BILL_AMT3,BILL_AMT1_DIV_BILL_AMT6,BILL_AMT5_DIV_PAY_AMT6,BILL_AMT1_DIV_BILL_AMT2,BILL_AMT5_DIV_PAY_AMT1,BILL_AMT2_DIV_PAY_AMT1,BILL_AMT3_DIV_BILL_AMT5,BILL_AMT2_DIV_PAY_AMT3,PAY_AMT1_DIV_PAY_AMT3,PAY_4_DIV_BILL_AMT1,BILL_AMT5_DIV_PAY_AMT4,BILL_AMT3_DIV_BILL_AMT6,BILL_AMT3_DIV_PAY_AMT3,BILL_AMT2_DIV_BILL_AMT5,BILL_AMT4_DIV_PAY_AMT3,PAY_AMT2_DIV_PAY_AMT5,PAY_AMT3_DIV_PAY_AMT4,PAY_AMT3_DIV_PAY_AMT6,LIMIT_BAL_DIV_BILL_AMT1,PAY_AMT2_DIV_PAY_AMT3,PAY_0_DIV_PAY_3,BILL_AMT2_DIV_BILL_AMT4
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10748,5.263112e+09,1.259058e+10,2.016973e+10,7.508475e+08,3.184500e+10,1.199325e+10,7.146426e+08,4.529821e+10,1.380849e+09,2.832307e+09,1.552555e+10,1.859663e+09,1.291795e+08,-49790.953141,7.776610e+08,8.431350e+09,1.452428e+09,1.987864e+10,1.177859e+09,6.953305e+07,7.275082e+07,6.685540e+07,1.070507e+11,1.090769e+08,0.001831,2.582497e+10,1.841392,1.299128,1.233107,21.780930,1.280380,4.196025,13.559640,1.110964,7.018103,14.395000,1.601964,21.923083,1.522965,0.000003,12.460963,2.949844,17.122319,2.051124,13.885509,2.017201,1.165848,1.268640,2.127194,1.285981,1.188680,1.578846
12574,1.681763e+09,1.890017e+09,2.106095e+09,2.284440e+08,2.290539e+09,2.075490e+09,2.198614e+08,2.536771e+09,2.396038e+08,2.818198e+08,1.947422e+09,3.952132e+08,4.606804e+07,-63042.590760,2.048174e+08,1.874032e+09,3.744260e+08,2.055538e+09,3.633889e+08,2.952766e+07,3.937973e+07,4.227221e+07,9.141954e+09,4.697003e+07,2.191545,2.223020e+09,1.039161,1.081477,1.030372,8.945970,1.055516,1.282439,7.947429,1.049247,7.292706,8.577580,1.114323,6.116832,0.713119,-0.000004,8.531057,1.157961,5.795108,1.176186,5.624288,1.156489,1.640410,1.528186,3.434579,0.727079,0.009337,1.087574
29677,3.495235e+09,3.755832e+09,8.375812e+09,4.372572e+09,4.635287e+09,4.330021e+09,2.788791e+08,2.574954e+09,2.405324e+08,2.838716e+08,3.927608e+09,3.669870e+08,4.246864e+07,-11539.235903,2.601676e+08,7.794661e+09,6.934625e+08,2.078528e+09,6.631335e+08,2.545531e+09,4.593547e+07,4.923919e+07,1.129207e+10,3.860490e+08,1.180640,4.432560e+09,0.503890,2.132521,1.045735,1.831740,0.529215,0.624234,6.314461,1.049699,7.320966,8.640031,2.230053,6.683497,0.773550,0.000015,6.768530,1.123702,12.629088,1.180176,12.076753,1.066372,1.195191,1.115012,4.177655,7.030644,0.064946,0.553418
8857,6.431053e+09,6.782168e+09,7.367865e+09,5.452921e+08,7.779446e+09,7.048416e+09,5.520514e+08,7.848470e+09,7.590732e+08,8.391804e+08,7.036828e+09,6.058977e+08,6.422549e+07,245047.946525,3.943163e+08,6.986428e+09,5.953889e+08,7.161031e+09,5.738420e+08,4.816747e+07,3.336326e+07,4.670929e+07,2.180503e+10,5.068938e+07,5.633307,7.497910e+09,1.007214,1.047043,1.037548,13.020932,1.017650,1.103902,11.731847,0.991375,8.532564,9.433020,1.086357,13.064387,1.384963,0.000043,16.423883,1.094194,12.837798,1.105532,12.373207,1.150323,1.389818,0.992770,2.802404,1.093101,0.780016,1.055861
21099,3.180829e+09,3.704580e+09,4.502759e+09,4.532627e+08,4.556503e+09,4.108747e+09,4.076556e+08,4.842404e+09,3.218626e+08,3.826143e+08,3.833020e+09,4.812895e+08,4.132234e+07,-16100.135379,2.751335e+08,3.866162e+09,4.921023e+08,3.748798e+09,4.756126e+08,6.763505e+07,3.532302e+07,5.233687e+07,3.171378e+10,7.097007e+07,-0.166074,4.403821e+09,0.991428,1.174726,1.034670,9.599974,0.978028,1.280644,7.734922,1.086620,9.796317,11.645341,1.215453,9.258114,0.795006,0.000011,11.459798,1.205034,9.466

In [0]:
inter_mult = inter_cal(list_one, list_two, train.drop(columns=[target]))

len one 26
len two 26


In [0]:
inter_mult

,BILL_AMT5_X_BILL_AMT6,BILL_AMT4_X_BILL_AMT5,BILL_AMT3_X_BILL_AMT4,BILL_AMT4_X_PAY_AMT5,BILL_AMT2_X_BILL_AMT3,BILL_AMT1_X_BILL_AMT6,BILL_AMT5_X_PAY_AMT6,BILL_AMT1_X_BILL_AMT2,BILL_AMT5_X_PAY_AMT1,BILL_AMT2_X_PAY_AMT1,BILL_AMT3_X_BILL_AMT5,BILL_AMT2_X_PAY_AMT3,PAY_AMT1_X_PAY_AMT3,PAY_4_X_BILL_AMT1,BILL_AMT5_X_PAY_AMT4,BILL_AMT3_X_BILL_AMT6,BILL_AMT3_X_PAY_AMT3,BILL_AMT2_X_BILL_AMT5,BILL_AMT4_X_PAY_AMT3,PAY_AMT2_X_PAY_AMT5,PAY_AMT3_X_PAY_AMT4,PAY_AMT3_X_PAY_AMT6,LIMIT_BAL_X_BILL_AMT1,PAY_AMT2_X_PAY_AMT3,PAY_0_X_PAY_3,BILL_AMT2_X_BILL_AMT4,BILL_AMT5_DIV_BILL_AMT6,BILL_AMT4_DIV_BILL_AMT5,BILL_AMT3_DIV_BILL_AMT4,BILL_AMT4_DIV_PAY_AMT5,BILL_AMT2_DIV_BILL_AMT3,BILL_AMT1_DIV_BILL_AMT6,BILL_AMT5_DIV_PAY_AMT6,BILL_AMT1_DIV_BILL_AMT2,BILL_AMT5_DIV_PAY_AMT1,BILL_AMT2_DIV_PAY_AMT1,BILL_AMT3_DIV_BILL_AMT5,BILL_AMT2_DIV_PAY_AMT3,PAY_AMT1_DIV_PAY_AMT3,PAY_4_DIV_BILL_AMT1,BILL_AMT5_DIV_PAY_AMT4,BILL_AMT3_DIV_BILL_AMT6,BILL_AMT3_DIV_PAY_AMT3,BILL_AMT2_DIV_BILL_AMT5,BILL_AMT4_DIV_PAY_AMT3,PAY_AMT2_DIV_PAY_AMT5,PAY_AMT3_DIV_PAY_AMT4,PAY_AMT3_DIV_PAY_AMT6,LIMIT_BAL_DIV_BILL_AMT1,PAY_AMT2_DIV_PAY_AMT3,PAY_0_DIV_PAY_3,BILL_AMT2_DIV_BILL_AMT4
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10748,818323977,4874987567,9312669875,84373000,16820818875,2446969836,115558000,26329934484,479276805,1264133115,6377357125,609588000,33180000,0,173337000,1563241125,441500000,8805346263,337492000,6000000,12000000,8000000,53559320000,24000000,0,12858192081,4.079356,1.460263,1.308176,84.289710,1.380717,12.198037,28.875562,1.133696,6.964802,18.370058,1.910280,38.089978,2.073482,0.000006,19.253582,7.792714,27.587103,2.637556,21.088228,5.995005,1.333222,1.999500,1.794268,1.499875,1.00,1.806220
12574,1058340,1923220,-1264120,307580,267784,29760,0,-11456,0,0,-851224,-1012424,0,-32,0,-695640,-2115344,-407404,4779320,0,0,0,320000,0,2,-605020,1.223416,1.484636,-0.441750,9.240437,0.477912,0.035446,1139.000000,-0.092437,1139.000000,-357.000000,-0.655838,-0.126193,0.000353,0.000000,1139.000000,-0.802363,-0.264051,-0.313433,0.597738,0.005464,2829.000000,2829.000000,303.060606,0.000353,-0.00,-0.211118
29677,57183100,59767500,2126665575,2022302325,0,18914410,1801800,0,0,0,60134100,0,0,0,1690000,2034706659,101765400,0,101145000,2034706659,2860000,3049200,21500000,101765400,1,0,0.029576,35.338970,1.006134,1.045194,0.000022,0.009798,0.937996,431.000000,1301.000000,1.000000,35.555726,0.000454,0.000454,0.002320,1.000000,1.051605,21.016811,0.000769,20.888687,1.051605,1.691776,1.586878,116.011601,21.016811,NaN,0.000022
8857,1616808312,1622422368,1634035548,65196800,1581904248,1488205943,63705600,1445729752,147319200,145957600,1596661416,63116800,5920000,109947,0,1628381307,64161600,1570661568,65196800,2560000,0,2560000,2931920000,2560000,6,1607427104,0.980521,1.023407,0.984122,25.452217,0.983717,0.902532,24.870081,0.929048,10.758444,10.659011,1.007158,24.640225,2.311680,0.000109,39817.000000,0.987539,25.048095,0.990758,25.452217,1.000000,1601.000000,1.000000,2.182838,1.000000,0.75,0.968097
21099,268643070,347719520,469701952,44896000,360478672,363838797,30980000,361426212,0,0,324112760,34456000,0,0,0,362884932,41848000,266861720,44896000,8000000,0,4000000,5664330000,8000000,0,386734144,0.893162,1.449164,0.932113,11.218891,0.823369,1.209640,7.741629,1.217714,15491.000000,17229.000000,1.350784,8.610195,0.000500,0.000048,15491.000000,1.206469,10.457271,1.112194,11.218891,1.999500,2001.000000,1.000000,12.869447,1.999500,2.00,0.767473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27956,585312,1270320,1226080,1106000,583552,564928,562800,583552,604608,565504,623904,1188160,1188160,-776,0,564928,1226080,604608,2496400,560000,0,1106000,108640000,1264000,-2,1188160,1.104252,1.963975,0.491461,2.255350,0.969112,1.065844,1.148359,1.031873,1.069057,1.000000,0.965217,0.476281,0.476281,0.000000,805.000000,1.065844,0.491461,0.935404

In [0]:
Second

In [0]:
 ungroup

,level_0,level_1,0
0,BILL_AMT5,BILL_AMT6,0.042100
1,BILL_AMT4,BILL_AMT5,0.040378
2,BILL_AMT3,BILL_AMT4,0.038016
3,BILL_AMT4,PAY_AMT5,0.035577
4,BILL_AMT2,BILL_AMT3,0.033015
...,...,...,...
248,LIMIT_BAL,SEX,0.000009
249,MARRIAGE,PAY_4,0.000008
250,SEX,PAY_AMT3,0.000007
251,SEX,BILL_AMT6,0.000006


In [0]:
inter.shape

(2000, 23, 23)

In [0]:
names = ["x","y","z"]
index = pd.MultiIndex.from_product([range(s)for s in inter.shape], names=names)
df = pd.DataFrame({'A': inter.flatten()}, index=index)['A']

In [0]:
df.shape

(1058000,)

In [0]:
df = df.unstack(level='x').swaplevel().sort_index()
df.columns = X
df.index.names = ['DATE', 'i']

ValueError: ignored

In [0]:
df.head()

x  y  z
0  0  0    0.722416
      1   -0.003709
      2   -0.011782
      3    0.005092
      4    0.010104
Name: A, dtype: float64

In [0]:
pan = pd.Panel(inter)
df = pan.swapaxes(0, 2).to_frame()
df.index = df.index.droplevel('minor')
df.index.name = 'Date'
df.index = df.index+1

TypeError: ignored

In [0]:
index = pd.MultiIndex.from_product([range(s)for s in A.shape], names=names)
df = pd.DataFrame({'A': A.flatten()}, index=index)['A']

In [0]:
inter

In [0]:
interact

array([[[ 7.22416088e-01, -3.70911867e-03, -1.17816884e-02, ...,
          5.45808674e-02, -1.61260361e-01,  0.00000000e+00],
        [-3.70911867e-03, -1.28327058e-03,  2.88120791e-03, ...,
          3.59514646e-03, -2.27297740e-03,  0.00000000e+00],
        [-1.17816884e-02,  2.88120791e-03, -5.94752639e-02, ...,
          5.10869690e-03, -2.38137691e-02,  0.00000000e+00],
        ...,
        [ 5.45808674e-02,  3.59514646e-03,  5.10869690e-03, ...,
          2.26046817e-01,  1.86530239e-02,  0.00000000e+00],
        [-1.61260361e-01, -2.27297740e-03, -2.38137691e-02, ...,
          1.86530239e-02,  1.02684490e-01,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00]],

       [[ 2.37102027e-01,  7.10941655e-03, -2.08144997e-02, ...,
         -7.74274108e-02,  5.69689024e-03,  0.00000000e+00],
        [ 7.10941655e-03,  3.34411610e-02, -7.23061904e-04, ...,
          6.69391521e-02,  3.91182175e

In [0]:
# the first 2,000 people in order to run quicker
# and then use 80% additive. 

shap_interaction_values = shap.TreeExplainer(model).shap_interaction_values(X.iloc[:2000,:])



In [0]:
# inter_mean = np.mean(inter,axis=0); inter_mean.shape
# inter_cross = pd.DataFrame(inter_mean, index=X, columns=X).abs()

# s = inter_cross.unstack()
# so = s.sort_values(kind="quicksort",ascending=False)

# def get_redundant_pairs(df):
#     '''Get diagonal and lower triangular pairs of correlation matrix'''
#     pairs_to_drop = set()
#     cols = df.columns
#     for i in range(0, df.shape[1]):
#         for j in range(0, i+1):
#             pairs_to_drop.add((cols[i], cols[j]))
#     return pairs_to_drop

# def get_top_abs_correlations(df):
#     au_corr = df.unstack()
#     labels_to_drop = get_redundant_pairs(df)
#     au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
#     return au_corr

# print("Top Absolute Correlations")
# get = get_top_abs_correlations(inter_cross)

# ungroup = get.to_frame().reset_index()
# ungroup = get.to_frame().reset_index()
# ungroup.groupby("level_0").mean().sort_values(by=0,ascending=False)

In [0]:
d_train = lgb.Dataset(train.drop([y],axis=1), label=train[y])
d_test = lgb.Dataset(test.drop([y],axis=1), label=test[y])
model = lgb.train(best_params, d_train, verbose_eval=1000)
y_pred_test = model.predict(test.drop([y],axis=1))
print('ROC AUC {}'.format(roc_auc_score(test[y], y_pred_test)))